# Q1. Notebook

What's the mean predicted duration for this dataset?

**16.19**

# Q2. Preparing the output

What's the size of the output file?

**19M**

# Q3. Creating the scoring script

Now let's turn the notebook into a script.
Which command you need to execute for that?

**jupyter nbconvert --to script hw4.ipynb**

# Q4. Virtual environment

What's the first hash for the Scikit-Learn dependency?

**"sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b"**

# Q5. Parametrize the script

What's the mean predicted duration?

**16.29**

# Q6. Docker contaner

Now run the script with docker. What's the mean predicted duration for April 2021?

**9.96**

# Python script

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import sklearn
import sys
import pickle
import pandas as pd


def read_data(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df


if __name__ == '__main__':

    year = int(sys.argv[1])
    month = int(sys.argv[2])
    taxi_type = 'fhv'

    input_file = f'/workspace/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
    output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

    with open('/app/model.bin', 'rb') as f_in:
        dv, lr = pickle.load(f_in)

    categorical = ['PUlocationID', 'DOlocationID']

    df = read_data(input_file)

    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = lr.predict(X_val)

    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

    df_result = df[['ride_id', 'duration']]

    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

    print(y_pred.mean())

# .dockerfile

In [ ]:
FROM agrigorev/zoomcamp-model:mlops-3.9.7-slim
RUN apt-get update && apt-get install -y 
WORKDIR /workspace
COPY requirements.txt /workspace/
RUN pip install -r requirements.txt
CMD '/bin/bash'

# requirements.txt

In [ ]:
numpy==1.22.4
pandas==1.4.2
scikit-learn==1.0.2
scipy==1.8.1
flask==2.1.2
pyarrow